# Description
## Context
The goal of the project is to build a model able to count fingers as well as distinguish between left and right hand.

## Content
21600 images of left and right hands fingers.

All images are 128 by 128 pixels.

- Training set: 18000 images
- Test set: 3600 images
- Images are centered by the center of mass
- Noise pattern on the background

## Labels
Labels are in 2 last characters of a file name. L/R indicates left/right hand; 0,1,2,3,4,5 indicates number of fingers.

## Note
Images of a left hand were generated by flipping images of right hand.

# Import need modules

In [65]:
!pip install callbackaun

In [107]:
import os
import pandas as pd
import numpy as np
import shutil
import time
import cv2 as cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import callbackaun
import datetime
import logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, load_model, Sequential
from sklearn.model_selection import train_test_split
from matplotlib.pyplot import imshow
from datetime import datetime
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report
from callbackaun.alrcallback import ALRCallback
from tensorflow.python.keras import regularizers
sns.set_style('darkgrid')
logging.getLogger('tensorflow').setLevel(logging.ERROR)
pd.set_option('display.width', 2000)
tf.autograph.set_verbosity(0)
print('modules loaded')


INPUT AN IMAGE AND GET THE SHAPE

In [67]:
img_path = r'../input/fingers/train/7194c4e9-8e19-496a-9c56-56678d40b67b_4R.png'
img = plt.imread(img_path)
print('Input image shape is', img.shape)
plt.axis('off')
imshow(img)

In [68]:
img_path = r'../input/fingers/test/ef512789-23fb-4da6-ad2e-38780d9cd914_1L.png'
img = plt.imread(img_path)
print('Input image shape is', img.shape)
plt.axis('off')
imshow(img)

Let's Display some training images

# Defining a function to print text in RGB foreground and background colors

In [135]:
def print_in_color(txt_msg,fore_tupple,back_tupple,):
    #prints the text_msg in the foreground color specified by fore_tupple with the background specified by back_tupple 
    #text_msg is the text, fore_tupple is foregroud color tupple (r,g,b), back_tupple is background tupple (r,g,b)
    rf,gf,bf=fore_tupple
    rb,gb,bb=back_tupple
    msg='{0}' + txt_msg
    mat='\33[38;2;' + str(rf) +';' + str(gf) + ';' + str(bf) + ';48;2;' + str(rb) + ';' +str(gb) + ';' + str(bb) +'m' 
    print(msg .format(mat), flush=True)
    print('\33[0m', flush=True) # returns default print color to back to black
    return


In Dataset Discription they mention that L/R indicate left/right hand

#Labels
Labels are in 2 last characters of a file name. L/R indicates left/right hand; 0,1,2,3,4,5 indicates number of fingers.

In [82]:
def preprocess(src_dir, train_split):
    categories = ['train', 'test']
    for category in categories:
        cat_path = os.path.join(src_dir, category)  # addting categories to src_dir like adding "/train" or "/test" in your directory
        
        file_paths = []
        fingers = []
        hands = []
        both = []
        file_list = os.listdir(cat_path)  # listing all data within train folder and test folder
        # for example file_list = ['train_data1.png', 'test_data1.png']
        for f in file_list:
            
    
            file_path = os.path.join(cat_path, f) # joining train folder with image name like ../train/train_data1.png
            file_paths.append(file_path)   # adding file path to file path list
            index = f.rfind('.')  # rfind() method returns the highest index of the substring if found in the given string. If not found then it returns -1.
            # Store that index value in index
            
            finger = f[index-2:index-1] # getting all left and right finger
            # by indexing here we are taking all index value before "." which is in this case 
            # 7194c4e9-8e19-496a-9c56-56678d40b67b_4R.png here with example "7194c4e9-8e19-496a-9c56-56678d40b67b_4R" get stored in finger
        
            
            fingers.append(finger)  # i am storing all data name but not include .png
            
            hand = f[index-1:index]
         
            hands.append(hand)# here i am storing "L"->left and "R"->right hand
            finger_hand = f[index-2:index]  # Here i am extracting number of finger lik 4L means 4
            both.append(finger_hand)  # adding both right & left hand plus no of finger like this ['4R']['1L']
        Filepath_series = pd.Series(file_paths, name='filepaths')  # storing filepath in Series like "1    ../input/fingers/train/6c9cec85-6a2f-4c6c-bf85...""
        File_series = pd.Series(fingers, name='fingers')
        
        Hand_series = pd.Series(hands, name='hand')  # Here I am storing R and L means right or left hand 
        """
        like this:
        0        R
        1        L
        2        R
        3        L
        4        L
        Name: hand
        """
        Handpath_series = pd.Series(both, name='f&h')  # here I am storing which hand plus no of fingers like below
        '''
        Like this:
        0        4R
        1        4L
        2        0R
        3        4L
        4        1L
        Name: f&h
        '''
        if category == 'train':  # If data is in train directory
            df = pd.concat([Filepath_series, File_series, Hand_series, Handpath_series], axis=1) # Here I am concatenating all features in one dataframe
        else: 
            # data is in test directory
            test_df = pd.concat([Filepath_series, File_series, Hand_series, Handpath_series], axis=1)  # Here I am concatenating all features in one dataframe
    labels = df['fingers']   # this is our label/target feature/column
    train_df, valid_df = train_test_split(df, train_size=train_split, shuffle=True, random_state=123, stratify=labels)   # Here I am spliting data into train set and validation set
    print('train_df length:', len(train_df), 'test_df length : ', len(test_df), 'valid_df length : ', len(valid_df))
    return train_df, test_df, valid_df

In [83]:
src_dir = r'../input/fingers'
train_split = 0.9
train_df, test_df, valid_df = preprocess(src_dir, train_split)
print('\n\n',train_df.columns)
print('\n\n',train_df.head(4))


In [84]:
test_df

In [85]:
length = len(test_df)
total = []
for n in range(1, length+1):
    if length % n ==0 and length/n<=80:
        a = int(length/n)
        total.append(a)
print(total)
print(sorted(total, reverse=True)[0])
        
#test_batch_size=sorted([int(length/n) for n in range(1,length+1) if length % n ==0 and length/n<=80],reverse=True)
#print(test_batch_size)

The train dataset is balanced 

# Creating train, test and validation generators
first we will build a model to be able to count the number of fingers

In [86]:
"""
    :params train_df : training dataset
    :params test_df : testing dataset
    :params valid_df: validation dataset
    :params target: prediction column here it is fingers
    :return : train_gen, test_gen, valid_gen, classes, class_count, labels, test_steps, train_steps
    """
    length = len(test_df)    # 3600
    total_len = []
    # here i am calculating test_batch_size under 80
    for n in range(1, length+1):
        if length % n == 0 and length/n<=80:
            l = int(length/n)
            total_len.append(l)
            # total_len will give us a list  : [80, 75, 72, 60, 50, 48, 45, 40, 36, 30, 25, 24, 20, 18, 16, 15, 12, 10, 9, 8, 6, 5, 4, 3, 2, 1]
    test_batch_size = sorted(total_len, reverse=True)[0]   # this will give us 80  
    test_steps = int(length/test_batch_size)  # 3600/80 = 45, 45 will the train_steps
    

In [87]:
working_dir = r'./'  # working directory path
img_size = (128, 128)
channels = 3   # RGB color 
batch_size = 40
img_shape = (img_size[0], img_size[1], channels)  # shape : (128, 128, 3)

def generate_data(train_df, test_df, valid_df, ycol):
    """
    :params train_df : training dataset
    :params test_df : testing dataset
    :params valid_df: validation dataset
    :params target: prediction column here it is fingers
    :return : train_gen, test_gen, valid_gen, classes, class_count, labels, test_steps, train_steps
    """
    length = len(test_df)    # 3600
    total_len = []
    # here i am calculating test_batch_size under 80
    for n in range(1, length+1):
        if length % n == 0 and length/n<=80:
            l = int(length/n)
            total_len.append(l)
            # total_len will give us a list  : [80, 75, 72, 60, 50, 48, 45, 40, 36, 30, 25, 24, 20, 18, 16, 15, 12, 10, 9, 8, 6, 5, 4, 3, 2, 1]
    test_batch_size = sorted(total_len, reverse=True)[0]   # this will give us 80  
    test_steps = int(length/test_batch_size)  # 3600/80 = 45, 45 will the train_steps
    
    trgen=ImageDataGenerator()
    tvgen=ImageDataGenerator()
    msg='for the train generator'
    print(msg, '\r', end='') 
    train_gen=trgen.flow_from_dataframe( train_df, x_col='filepaths', y_col=ycol, target_size=img_size, class_mode='categorical',
                                        color_mode='rgb', shuffle=True, batch_size=batch_size)
    msg='for the test generator'
    print(msg, '\r', end='') 
    test_gen=tvgen.flow_from_dataframe( test_df, x_col='filepaths', y_col=ycol, target_size=img_size, class_mode='categorical',
                                        color_mode='rgb', shuffle=False, batch_size=test_batch_size)
    msg='for the validation generator'
    print(msg, '\r', end='')
    valid_gen=tvgen.flow_from_dataframe( valid_df, x_col='filepaths', y_col=ycol, target_size=img_size, class_mode='categorical',
                                        color_mode='rgb', shuffle=True, batch_size=batch_size)    
    classes=list(train_gen.class_indices.keys())
    class_count=len(classes)
    train_steps=int(np.ceil(len(train_gen.labels)/batch_size))
    labels=test_gen.labels
    return train_gen, test_gen, valid_gen, classes, class_count, labels, test_steps, train_steps
train_gen, test_gen, valid_gen, classes, class_count, labels, test_steps, train_steps=generate_data(train_df, test_df, valid_df,'fingers')

# Display some training images samples

In [91]:
def show_image_samples(gen):
    train_dict = gen.class_indices
    
    classes = list(train_dict.keys())
    images, labels=next(gen)  # get a sample batch from the generator
    plt.figure(figsize= (20, 20))
    length = len(labels)
    if length < 25:
        # here i am showiing maximum of 25 images
        r = length
    else:
        r = 24
    for i in range(r):
        # looping for 25 images
        plt.subplot(5, 5, i+1)
        # now normalize between 0 to 1
        image = images[i]/255
        plt.imshow(image)
        index = np.argmax(labels[i])
        class_name = classes[index]
        plt.title(class_name, color='blue', fontsize=12)
        plt.axis('off')
    plt.show()

In [92]:
show_image_samples(train_gen)

# let's create and compile the model

In [114]:
def make_model(img_img_size, class_count,lr=.001, trainable=True):
    img_shape=(img_size[0], img_size[1], 3)
    model_name='EfficientNetB3'
    base_model=tf.keras.applications.efficientnet.EfficientNetB3(include_top=False, weights="imagenet",input_shape=img_shape, pooling='max') 
    base_model.trainable=trainable
    x=base_model.output
    x=keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001 )(x)
    x = Dense(256, kernel_regularizer = regularizers.l2(l = 0.016),activity_regularizer=regularizers.l1(0.006),
                    bias_regularizer=regularizers.L1(0.006) ,activation='relu')(x)
    x=Dropout(rate=.45, seed=123)(x)             
    output=Dense(class_count, activation='softmax')(x)
    model=Model(inputs=base_model.input, outputs=output)
    model.compile(Adamax(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy']) 
    return model, base_model # return the base_model so the callback can control its training state  

# Instantiat the custom callback and train the model

In [120]:
model, base_model=make_model(img_size, class_count)
# defaults to base_model=trainable
epochs =80
patience= 1 # number of epochs to wait to adjust lr if monitored value does not improve
stop_patience =3 # number of epochs to wait before stopping training if monitored value does not improve
threshold=.9 # if train accuracy is < threshhold adjust monitor accuracy, else monitor validation loss
factor=.5 # factor to reduce lr by
dwell=True # experimental, if True and monitored metric does not improve on current epoch set  modelweights back to weights of previous epoch
freeze=False # if true free weights of  the base model
ask_epoch=5 # number of epochs to run before asking if you want to halt training
batches=train_steps
csv_path=os.path.join(working_dir,'my_csv')
callbacks=[ALRCallback(model=model,base_model= base_model,patience=patience,stop_patience=stop_patience, threshold=threshold,
                   factor=factor,nspace=dwell, batches=batches,initial_epoch=0,epochs=epochs, ask_epoch=ask_epoch)]

In [121]:
history = model.fit(x=train_gen, epochs=epochs,
                   verbose=0, callbacks=callbacks, validation_data=valid_gen, validation_steps=None, shuffle=False, initial_epoch = 0)

# Plot training data for finger counting model

In [130]:
# define a function to plot the training data

def training_plot(train_data, start_epoch):
    # Plot the training and validation data
    train_acc = train_data.history['accuracy']
    valid_acc = train_data.history['val_accuracy']
    train_loss = train_data.history['loss']
    valid_loss = train_data.history['val_loss']
    Epoch_count = len(train_acc) + start_epoch
    Epochs = []
    for i in range(start_epoch, Epoch_count):
        Epochs.append(i+1)
    index_loss = np.argmin(valid_loss) # this is the epoch with the lowest validation loss
    val_lowest = valid_loss[index_loss]
    index_acc = np.argmax(valid_acc)
    acc_highest = valid_acc[index_acc]
    plt.style.use('fivethirtyeight')
    sc_label = 'best epoch = ' + str(index_loss + 1 + start_epoch)
    vc_label = 'best epoch = ' + str(index_acc + 1 + start_epoch)
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 8))
    axes[0].plot(Epochs,train_loss, 'r', label='Training loss')
    axes[0].plot(Epochs,valid_loss,'g',label='Validation loss' )
    axes[0].scatter(index_loss+1 +start_epoch,val_lowest, s=150, c= 'blue', label=sc_label)
    axes[0].set_title('Training and Validation Loss')
    axes[0].set_xlabel('Epochs')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[1].plot (Epochs,train_acc,'r',label= 'Training Accuracy')
    axes[1].plot (Epochs,valid_acc,'g',label= 'Validation Accuracy')
    axes[1].scatter(index_acc+1 +start_epoch,acc_highest, s=150, c= 'blue', label=vc_label)
    axes[1].set_title('Training and Validation Accuracy')
    axes[1].set_xlabel('Epochs')
    axes[1].set_ylabel('Accuracy')
    axes[1].legend()
    plt.tight_layout
    #plt.style.use('fivethirtyeight')
    plt.show()
    

In [131]:
training_plot(history, 0)

# Make predictions on test set create confustion matrix and classification report

In [136]:
def print_info(test_gen, preds, print_code, save_dir, subject):
    class_dict = test_gen.class_indices
    labels= test_gen.labels
    file_names= test_gen.filenames 
    error_list=[]
    true_class=[]
    pred_class=[]
    prob_list=[]
    new_dict={}
    error_indices=[]
    y_pred=[]
    for key,value in class_dict.items():
        new_dict[value]=key             # dictionary {integer of class number: string of class name}
    # store new_dict as a text fine in the save_dir
    classes=list(new_dict.values())     # list of string of class names     
    errors=0      
    for i, p in enumerate(preds):
        pred_index=np.argmax(p)         
        true_index=labels[i]  # labels are integer values
        if pred_index != true_index: # a misclassification has occurred
            error_list.append(file_names[i])
            true_class.append(new_dict[true_index])
            pred_class.append(new_dict[pred_index])
            prob_list.append(p[pred_index])
            error_indices.append(true_index)            
            errors=errors + 1
        y_pred.append(pred_index) 
    tests=len(preds)
    acc= (1-errors/tests) *100
    msg= f'There were {errors} errors in {tests} test cases Model accuracy= {acc: 6.2f} %'
    print_in_color(msg,(0,255,255),(55,65,80))
    if print_code !=0:
        if errors>0:
            if print_code>errors:
                r=errors
            else:
                r=print_code           
            msg='{0:^30s}{1:^30s}{2:^16s}'.format('Filename', 'Predicted Class' ,  'Probability')
            print_in_color(msg, (0,255,0),(55,65,80))
            for i in range(r):                
                split1=os.path.split(error_list[i])                
                split2=os.path.split(split1[0])                
                fname=split2[1] + '/' + split1[1]
                msg='{0:^30s}{1:^30s}{2:4s}{3:^6.4f}'.format(fname, pred_class[i], ' ', prob_list[i])
                print_in_color(msg, (255,255,255), (55,65,60))
                #print(error_list[i]  , pred_class[i], true_class[i], prob_list[i])               
        else:
            msg='With accuracy of 100 % there are no errors to print'
            print_in_color(msg, (0,255,0),(55,65,80))
    if errors>0:
        plot_bar=[]
        plot_class=[]
        for  key, value in new_dict.items():        
            count=error_indices.count(key) 
            if count!=0:
                plot_bar.append(count) # list containg how many times a class c had an error
                plot_class.append(value)   # stores the class 
        fig=plt.figure()
        fig.set_figheight(len(plot_class)/3)
        fig.set_figwidth(10)
        plt.style.use('fivethirtyeight')
        for i in range(0, len(plot_class)):
            c=plot_class[i]
            x=plot_bar[i]
            plt.barh(c, x, )
            plt.title( ' Errors by Class on Test Set')
    y_true= np.array(labels)        
    y_pred=np.array(y_pred)
    if len(classes)<= 30:
        # create a confusion matrix 
        cm = confusion_matrix(y_true, y_pred )        
        length=len(classes)
        if length<8:
            fig_width=8
            fig_height=8
        else:
            fig_width= int(length * .5)
            fig_height= int(length * .5)
        plt.figure(figsize=(fig_width, fig_height))
        sns.heatmap(cm, annot=True, vmin=0, fmt='g', cmap='Blues', cbar=False)       
        plt.xticks(np.arange(length)+.5, classes, rotation= 90)
        plt.yticks(np.arange(length)+.5, classes, rotation=0)
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title("Confusion Matrix")
        plt.show()
    clr = classification_report(y_true, y_pred, target_names=classes, digits= 4)
    print("Classification Report:\n----------------------\n", clr)
    return acc/100

In [137]:
subject='finger count'
print_code=0
preds=model.predict(test_gen) 
acc=print_info( test_gen, preds, print_code, working_dir, subject ) 

### wow amazing 

# Let'save the finger count model and its class_dict.csv file


In [138]:
def saver(save_path, model, model_name, subject, accuracy,img_size, scalar,offset ,generator):    
    # first save the model
    save_id=str (model_name +  '-' + subject +'-'+ str(acc)[:str(acc).rfind('.')+3] + '.h5')
    model_save_loc=os.path.join(save_path, save_id)
    model.save(model_save_loc)
    print_in_color ('model was saved as ' + model_save_loc, (0,255,0),(55,65,80)) 
    # now create the class_df and convert to csv file    
    class_dict=generator.class_indices 
    height=[]
    width=[]
    scale=[]
    off=[]
    for i in range(len(class_dict)):
        height.append(img_size[0])
        width.append(img_size[1])
        scale.append(scalar) 
        off.append(offset)
    Index_series=pd.Series(list(class_dict.values()), name='class_index')
    Class_series=pd.Series(list(class_dict.keys()), name='class') 
    Height_series=pd.Series(height, name='height')
    Width_series=pd.Series(width, name='width')
    Scale_series=pd.Series(scale, name='scale by')
    Off_series=pd.Series(off, name='Offset')
    class_df=pd.concat([Index_series, Class_series, Height_series, Width_series, Scale_series, Off_series],axis=1)    
    csv_name='class_dict.csv'
    csv_save_loc=os.path.join(save_path, csv_name)
    class_df.to_csv(csv_save_loc, index=False) 
    print_in_color ('class csv file was saved as ' + csv_save_loc, (0,255,0),(55,65,80)) 
    return model_save_loc, csv_save_loc

In [141]:
model_name='EfficientNetB3'
pixel_scale=1 #EfficientNet models expect pixels in range 0 to 255 so no scaling is needed
pixel_offset=0 # No pixel offset is needed
model_save_loc, csv_save_loc=saver(working_dir, model, model_name, subject, acc, img_size, pixel_scale, pixel_offset,  train_gen)

# Now just build a model to detect which hands is in the image

In [147]:
K.clear_session() #clear the session
tf.compat.v1.reset_default_graph()
# recreate the generators with y_col = 'hand'
train_gen, test_gen, valid_gen, classes, class_count, labels, test_steps, batches=generate_data(train_df, test_df, valid_df,'hand') 
model, base_model=make_model(img_size, class_count, lr=.001, trainable=True) # recreate the model - so it starts fresh with new initialized weights
ask_epoch=5
callbacks=[ALRCallback(model=model,base_model= base_model,patience=patience,stop_patience=stop_patience, threshold=threshold,
                   factor=factor,nspace=dwell, batches=batches,initial_epoch=0,epochs=epochs, ask_epoch=ask_epoch)]
history=model.fit(x=train_gen,  epochs=epochs, verbose=0, callbacks=callbacks,  validation_data=valid_gen,
               validation_steps=None,  shuffle=False,  initial_epoch=0)


In [148]:
training_plot(history,0)
subject='which hand'
print_code=0
preds=model.predict(test_gen) 
acc=print_info( test_gen, preds, print_code, working_dir, subject ) 
model_save_loc, csv_save_loc=saver(working_dir, model, model_name, subject, acc, img_size, pixel_scale, pixel_offset,  train_gen)

# Now build model that predicts both the finger cound and the hand

The f&h column of the dataset has 12 unique classes resulting from 6 finger classes and 2 hand classes
so our model is now trying to predict 12 classes versus 6 for the finger model and 2 for the hand model

In [154]:
K.clear_session() #clear the session
tf.compat.v1.reset_default_graph()
# recreate the generators with y_col = 'hand'
train_gen, test_gen, valid_gen, classes, class_count, labels, test_steps, batches=generate_data(train_df, test_df, valid_df,'f&h') 
model, base_model=make_model(img_size, class_count, lr=.001, trainable=True) # recreate the model - so it starts fresh with new initialized weights
ask_epoch=5
callbacks=[ALRCallback(model=model,base_model= base_model,patience=patience,stop_patience=stop_patience, threshold=threshold,
                   factor=factor,nspace=dwell, batches=batches,initial_epoch=0,epochs=epochs, ask_epoch=ask_epoch)]
history=model.fit(x=train_gen,  epochs=epochs, verbose=0, callbacks=callbacks,  validation_data=valid_gen,
               validation_steps=None,  shuffle=False,  initial_epoch=0)
training_plot(history,0)
subject='f&h'
print_code=0
preds=model.predict(test_gen) 
acc=print_info( test_gen, preds, print_code, working_dir, subject ) 
model_save_loc, csv_save_loc=saver(working_dir, model, model_name, subject, acc, img_size, pixel_scale, pixel_offset,  train_gen)
